In [27]:
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from icecream import ic

In [2]:
class FabricPhysicalConstants:
    rho= 2300 #kg/m^3 
    cp= 750 # J/(kg-K) 
    k=0.8 # W/(m-K)
    h = 4 # W/(m-K)
    T0= 293.15 # K 

    alpha = k/(cp*rho) # thermal diffusivity

In [3]:
f = FabricPhysicalConstants()
f.alpha

4.6376811594202904e-07

In [4]:
# in example => th = 0.10 m, dt = 0.010 m 
thickness = 0.10 
dx = 0.010

In [5]:
x_vals = np.arange(start=0, stop=thickness, step=dx)
len(x_vals)

10

In [21]:
dt = 15 # seconds
period = dt * 4 * 60/20 # 1 hour/ 
times = np.arange(start=0, stop=period , step=dt)
len(times)
# TODO resolve difference between this time formulation and that of indoor air => ALSO need to do check of tau 

12

In [22]:
# initialize 
Txt = np.zeros((len(times), len(x_vals)))
Txt[0,:] = f.T0
Txt.shape

(12, 10)

In [23]:
tau = f.alpha * dt / (dx**2)
tau
Ta =  np.zeros(len(x_vals))
Tb = np.zeros(len(x_vals))

# Ta = Tb = x => calling these the same pobject => pointing to the same place 

In [31]:
for i, _ in enumerate(times):
    row = Txt[i,:] # row at a given timestep
    for m, _ in enumerate(Txt[i,:]):
        m = m + 1
        if m < len(x_vals) - 1:
            Ta[m] = row[m - 1] + row[m + 1]
            Tb[m] = row[m]

    if i < len(times) - 1:
        Txt[i+1, :] = Ta*tau + Tb*(1 - 2*tau)

In [32]:
Txt

array([[  0.        , 239.98930632, 289.2888143 , 293.05131155,
        293.15      , 293.15      , 293.05131155, 289.2888143 ,
        239.98930632,   0.        ],
       [  0.        , 226.72392904, 286.12102225, 292.79643789,
        293.14313472, 293.14313472, 292.79643789, 286.12102225,
        226.72392904,   0.        ],
       [  0.        , 215.08380133, 282.45342729, 292.35617919,
        293.11901668, 293.11901668, 292.35617919, 282.45342729,
        215.08380133,   0.        ],
       [  0.        , 204.80803261, 278.4557317 , 291.72035906,
        293.06594972, 293.06594972, 291.72035906, 278.4557317 ,
        204.80803261,   0.        ],
       [  0.        , 195.6838355 , 274.25517019, 290.89120868,
        292.97234341, 292.97234341, 290.89120868, 274.25517019,
        195.6838355 ,   0.        ],
       [  0.        , 187.53687892, 269.94662785, 289.87869364,
        292.82756882, 292.82756882, 289.87869364, 269.94662785,
        187.53687892,   0.        ],
       [  

In [15]:
Txt[1]

array([  0.        , 272.75695652, 293.15      , 293.15      ,
       293.15      , 293.15      , 293.15      , 293.15      ,
       272.75695652,   0.        ])

In [16]:
Txt[2]

array([  0.        , 255.20120605, 291.7313535 , 293.15      ,
       293.15      , 293.15      , 293.15      , 291.7313535 ,
       255.20120605,   0.        ])

In [17]:
Txt[3]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
row_num = 0
row = Txt[row_num,:]
for ix, _ in enumerate(Txt[row_num,:]):
    ix = ix + 1
    if ix < len(x_vals) - 1:
        Ta[ix] = row[ix - 1] + row[ix + 1]
        Tb[ix] = row[ix]

print(Ta, Tb)

Txt[1, :] = Ta*tau + Tb*(1 - 2*tau)

Txt[1, :]
    

[  0.  586.3 586.3 586.3 586.3 586.3 586.3 586.3 586.3   0. ] [  0.   293.15 293.15 293.15 293.15 293.15 293.15 293.15 293.15   0.  ]


array([  0.  , 293.15, 293.15, 293.15, 293.15, 293.15, 293.15, 293.15,
       293.15,   0.  ])

In [11]:
row_num = 1
row = Txt[row_num,:]
for ix, _ in enumerate(Txt[row_num,:]):
    ix = ix + 1
    if ix < len(x_vals) - 1:
        Ta[ix] = row[ix - 1] + row[ix + 1]
        Tb[ix] = row[ix]

print(Ta, Tb)

Txt[row_num+1, :] = Ta*tau + Tb*(1 - 2*tau)

Txt[row_num+1, :]

[  0.   293.15 586.3  586.3  586.3  586.3  586.3  586.3  293.15   0.  ] [  0.   293.15 293.15 293.15 293.15 293.15 293.15 293.15 293.15   0.  ]


array([  0.        , 272.75695652, 293.15      , 293.15      ,
       293.15      , 293.15      , 293.15      , 293.15      ,
       272.75695652,   0.        ])